<a href="https://colab.research.google.com/github/cesarniyo/Competition/blob/master/FellowAI_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FellowAI Competition 

*by Cesar Niyomugabo:Fellow Team Austin*
     
     
 ## Property  analysis approach     
 
 
 In this project we combined our trained model on flooring damage/ decay detection with other team's models to detect any damaged floors, overgrown plants, exposed wires and electrical unsafely handled. Our detailed approach is explained in following steps: 
 


**Step 1: Download  the videon from youtube**

On this step we do access the video from the  Youtube link provided


**Step 2: Reading in the video and feed frames one by one to the models for predictions.**

- **Flooring model**

 >The flooring model predicts two classes (Damaged and Normal). If Damage detected, result are displayed on the frame, otherwise nothing displayed.


-  **Landscape model**:

 >The Landscape model predicts two categories (clean, overgrown).If overgrown landscape detected, we display the result on the frame, if  'clean' predicted nothing displayed


- **childproofed-electric model**:

 >The childproofed-electric model predicts 3 categories (childproofed, open plugs, open wires).If open wires predicted with model confidence over 98% we display the result on the frame, otherwise nothing detected.

- **electrical safety model**:

 >The electrical safely model predicts 2 categories (unsafe, safe).If unsafe predicted with model confidence over 95%,  we display the result on the frame, otherwise nothing detected.
 
 



**Step 3: Prediction result analysis**

On this step we compare the models prediction results to real content of the frame to get the strength of each model(Detailed in the conclusion)


## Downloading the video from youtube

In [0]:

!pip install pytube
import pytube
video_url='https://www.youtube.com/watch?v=mdHm8hAwVUA&feature=youtu.be'
youtube=pytube.YouTube(video_url)#Pass the url here
video=youtube.streams.first()
!mkdir '/content/video'          # create a folder to hold the video on google colab
video.download('/content/video') # Path, where to video download


mkdir: cannot create directory ‘/content/video’: File exists


'/content/video/Arshak’s house of defects.mp4'

## Importing the libraries

In [0]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from fastai.vision import *
#from PIL import Image

## Unzipping the folder containing the models

In [0]:

from zipfile import ZipFile
file_name= "models.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')


Done


##  Video Processing (Prediction)

In [0]:

def prediction(frame,path):  
       learn = load_learner(path)
       learn = learn.to_fp32()
       frame_copy =  frame.copy()
       frame_resized = cv2.resize(frame_copy,(224,224))        
       t = torch.tensor(np.ascontiguousarray(np.flip( frame_resized, 2)).transpose(2,0,1)).float()/255
       img = Image(t) # fastai.vision.Image, not PIL.Image       
       p = learn.predict(img) #pred_class,pred_idx,outputs = learn.predict(img)
       #print(p[0])
       return p  
  
 
  
  
def VideoProcessing(video,path1,path2,path3,path4): 
  
      # Create a VideoCapture object and read from input file
      # If the input is the camera, pass 0 instead of the video file name
      cap = cv2.VideoCapture(video)

      # Check if camera opened successfully
      if (cap.isOpened()== False): 
          print("Error opening video stream or file")

      # Default resolutions of the frame are obtained.The default resolutions are system dependent.
      # We convert the resolutions from float to integer.
      frame_width = int(cap.get(3))
      frame_height = int(cap.get(4))
      print('...video Processing in Progress')
      
      x,y=0,620
      w=300
      h=50

      # Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
      out = cv2.VideoWriter('AustinTeam.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

      
      #Read until video is completed
      while(cap.isOpened()):
          # Capture frame-by-frame
          ret, frame = cap.read()
          if ret == True:
            
              p1=prediction(frame,path1)#flooring
              p2=prediction(frame,path2)#overgrownplants       
              p3=prediction(frame,path3)#childproofelectrical  
              p4=prediction(frame,path4)#electrical
              #print(p1)



                
              # Write the frame into the file 'output.avi'
              if str(p1[0])=='Damaged':
                 cv2.rectangle(frame,(0, 620),(300,590),(0,255,0),4)#cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),4)
                 #cv2.putText(image, 'name', starting point, font, fontScale, color, thickness, cv2.LINE_AA)
                 cv2.putText(frame, 'Damaged-Floor', (0, 620), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255), 2, cv2.LINE_AA, False)
                 out.write(frame)

              if str(p2[0])=='overgrown': 
                 cv2.rectangle(frame,(0,520),(300,490),(0,255,0),4)
                 cv2.putText(frame, 'overgrown-plants', (0,520), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255), 2, cv2.LINE_AA, False)
                 out.write(frame)


              if str(p3[0])=='open wires' and p3[2][2]>=0.98 : #childproofed #open plugs #open wires
                text = f'{p3[0]}'
                cv2.rectangle(frame,(0,420),(300,390),(0,255,0),4)#cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),4)
                cv2.putText(frame, text, (0,420), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255), 2, cv2.LINE_AA, False)
                out.write(frame)

              if str(p4[0])=='unsafe'and p4[2][1]>=0.95 : 
                cv2.rectangle(frame,(0,320),(300,290),(0,255,0),4)
                cv2.putText(frame, 'Exposed-electric', (0,320), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 0, 255), 2, cv2.LINE_AA, False)
                out.write(frame)


              else:
                 out.write(frame)
                 #pass 

              # Write the frame into the file 'austin.avi' 
              #out.write(frame)

              # Display the resulting frame
              #cv2.imshow('frame',gray)

              # Press Q on keyboard to  exit
              if cv2.waitKey(25) & 0xFF == ord('q'):
                 break

          # Break the loop
          else: 
            break

      # When everything done, release the video capture object
      cap.release()
      out.release() 

      # Closes all the frames
      cv2.destroyAllWindows()
      return ('Done')

In [0]:

path1= '/content/models/flooring'
path2= '/content/models/overgrownplants'
path3= '/content/models/childproofelectrical'
path4= '/content/models/electrical'


video='/content/video/Arshak’s house of defects.mp4'
VideoProcessing(video,path1,path2,path3,path4)


...video Processing in Progress


'Done'

## Conclusion



 We have combined our model with models from different teams to detect any damage/risk in and around a house. The models we experimented with were able to yeild some good results however we also noticed some short comings in classifying the image properly, bellow are some  of the observation details:

**Flooring model**

*   True Positives observed :Yes
*   False Positives observed :Yes, floors with shadow or differenty lighting are classified as damaged, also floors with stairs at edges are classified as damage as well.




**Landscape model**:


*   True Positives observed:Yes
*   False Positives observed:Yes, this model parfomed well except in some cases were it shows true positives inside the house



**childproofed-electric model**:

*   True Positives observed:Yes
*   False Positives observed:Yes, small tree truncs are classified as open wires



**electrical safety model**:

*   True Positives observed:Yes
*   False Positives observed:Yes, classifying trees as electric hazard.



From the above observations we come to the conclusion that the main problem lies on how the models were trained individually earlier. For example, our flooring  damage model, - although we had several damage categories and their respective 'Normal ' categories, most of them did note have the virition of the light and shadows like seen in the video. So, the model sometime fails to classify Normal floor collectly if any shodows or different light intensity on it.  We need to have larger dataset also consider some of the complex cases to train a better model.




Notice:some of other team's models we tried to use were note able to be loaded as pickles,so note able to experiment with those.
